In [2]:
import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior() 

from tensorflow import math
import pandas as pd
import numpy as np
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Data process for removing cold start issue

In [6]:
df = pd.read_csv('train_counted_original.csv')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163345 entries, 0 to 163344
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   overall               163345 non-null  int64 
 1   reviewTime            163345 non-null  object
 2   reviewerID            163345 non-null  object
 3   reviewerID_int        163345 non-null  int64 
 4   reviewText            163345 non-null  object
 5   summary               163345 non-null  object
 6   unixReviewTime        163345 non-null  int64 
 7   category              163345 non-null  object
 8   price                 163345 non-null  object
 9   itemID                163345 non-null  object
 10  itemID_int            163345 non-null  int64 
 11  reviewHash            163345 non-null  int64 
 12  reviewerID_int_count  163345 non-null  int64 
 13  itemID_int_count      163345 non-null  int64 
dtypes: int64(7), object(7)
memory usage: 17.4+ MB


,overall,reviewTime,reviewerID,reviewerID_int,reviewText,summary,unixReviewTime,category,price,itemID,itemID_int,reviewHash,reviewerID_int_count,itemID_int_count
0,4,"08 24, 2010",u04428712,3227,"So is Katy Perry's new album ""Teenage Dream"" c...",Amazing that I Actually Bought This...More Ama...,1282608000,Pop,$35.93,p70761125,18183,85559980,11,70
1,5,"10 31, 2009",u06946603,5054,"I got this CD almost 10 years ago, and given t...",Excellent album,1256947200,Alternative Rock,$11.28,p85427891,21852,41699565,10,39
2,4,"10 13, 2015",u92735614,67076,I REALLY enjoy this pairing of Anderson and Po...,"Love the Music, Hate the Light Show",1444694400,Pop,$89.86,p82172532,21020,24751194,2,23
3,5,"06 28, 2017",u35112935,25508,Finally got it . It was everything thought it ...,Great,1498608000,Pop,$11.89,p15255251,3900,22820631,1,49
4,5,"09 7, 2015",u07624734,5549,o.k.,Five Stars,1441584000,Pop,$14.99,p78489708,20107,23609516,4,15


In [7]:
review_short = df[df['reviewerID_int_count'] <= 10]
review_short

,overall,reviewTime,reviewerID,reviewerID_int,reviewText,summary,unixReviewTime,category,price,itemID,itemID_int,reviewHash,reviewerID_int_count,itemID_int_count
1,5,"10 31, 2009",u06946603,5054,"I got this CD almost 10 years ago, and given t...",Excellent album,1256947200,Alternative Rock,$11.28,p85427891,21852,41699565,10,39
2,4,"10 13, 2015",u92735614,67076,I REALLY enjoy this pairing of Anderson and Po...,"Love the Music, Hate the Light Show",1444694400,Pop,$89.86,p82172532,21020,24751194,2,23
3,5,"06 28, 2017",u35112935,25508,Finally got it . It was everything thought it ...,Great,1498608000,Pop,$11.89,p15255251,3900,22820631,1,49
4,5,"09 7, 2015",u07624734,5549,o.k.,Five Stars,1441584000,Pop,$14.99,p78489708,20107,23609516,4,15
5,5,"06 25, 2016",u64810771,47001,Bought this album while sitting at a Boney Jam...,Bought this album while sitting at a Boney Jam...,1466812800,Jazz,$8.73,p58524163,14971,11806672,3,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163339,5,"11 15, 2014",u85625112,61920,Excellent,Five Stars,1416009600,Alternative Rock,$9.95,p61111203,15659,47089196,4,16
163341,4,"05 1, 2004",u68902609,49926,"With this, Mariah's third album, Mariah proved...",Well Done Mariah! You Show 'Em!,1083369600,Pop,$7.98,p84118731,21491,35077372,5,17
163342,3,"03 1, 2011",u25124021,18250,"This recording is rather disappointing, to a c...",Odd Couplings,1298937600,Classical,$13.57,p25341819,6395,71627957,3,8
163343,5,"03 20, 2016",u04485604,3268,Get it now ! Right now ! I am partial. I am a ...,Our Poet,1458432000,Alternative Rock,$11.07,p19134748,4873,27463540,4,44


In [9]:
review_long = df[df['reviewerID_int_count'] > 10]
review_long

,overall,reviewTime,reviewerID,reviewerID_int,reviewText,summary,unixReviewTime,category,price,itemID,itemID_int,reviewHash,reviewerID_int_count,itemID_int_count
0,4,"08 24, 2010",u04428712,3227,"So is Katy Perry's new album ""Teenage Dream"" c...",Amazing that I Actually Bought This...More Ama...,1282608000,Pop,$35.93,p70761125,18183,85559980,11,70
26,2,"09 6, 2006",u01301388,920,"Well, I know that Ministry's career was/is sup...",We've heard it before...,1157500800,Alternative Rock,$2.99,p07016047,1822,73559038,43,10
32,5,"11 3, 2010",u42004673,30557,"Four discs 74,73,49,43 minutes each approximat...",SO MUCH FINE MUSIC WHERE DO YOU START?,1288742400,Pop,$29.31,p68214195,17540,84832082,60,5
35,5,"10 28, 2002",u31093491,22563,"""Kick"" sold more copies, but ""Listen Like Thie...",XS-ively Good.,1035763200,Alternative Rock,$9.99,p70126988,18035,98962224,134,20
40,5,"05 16, 2014",u16807919,12143,"Lou Reed, Berlin. Easily one of the best album...",Lou Reed - Berlin,1400198400,Alternative Rock,$6.64,p49109709,12492,28053917,19,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163319,5,"06 23, 2004",u43304033,31548,"She's singin' like a Billie Holiday, and Red H...",Billie Holiday of modern days!,1087948800,Jazz,$2.77,p73686351,18942,41630847,13,8
163321,4,"12 1, 1999",u43323645,31562,The golden age of British metal is well repres...,THE GOLDEN AGE !,944006400,Alternative Rock,$6.00,p91436560,23292,91611373,28,7
163323,5,"03 5, 2016",u84835853,61370,It's an album I thoroughly enjoy listening to ...,It's All About 1989,1457136000,Pop,$12.49,p98884595,25218,35659195,13,246
163330,4,"02 12, 2006",u97675799,70603,"The truth is, when ""Galore"" was released, it w...","The singles, volume 2.",1139702400,Alternative Rock,$17.33,p89289226,22780,80665701,206,19


In [10]:
df_val = pd.DataFrame(columns=['overall', 'reviewerID_int', 'itemID_int', 'reviewerID_int_count'])
df_train = review_long.copy()

for i in np.unique(review_long['reviewerID_int']):
    df_temp = review_long[review_long['reviewerID_int'] == i]
    df_temp1 = df_temp.sample(1)
    df_val = df_val.append(df_temp1)
    for index, rows in df_temp1.iterrows():
        df_train.drop(index,inplace=True)

In [11]:
df_val.to_json(r'./p2TestingLongv2.json', orient='records', lines=True)
df_train.to_json(r'./p2TrainingLongv2.json', orient='records', lines=True)

In [12]:
train_short, test_short = train_test_split(review_short, test_size = 0.2, random_state=42)
test_short

,overall,reviewTime,reviewerID,reviewerID_int,reviewText,summary,unixReviewTime,category,price,itemID,itemID_int,reviewHash,reviewerID_int_count,itemID_int_count
163088,4,"01 26, 2005",u23440094,17017,I doubted long between giving this the perfect...,Kid A meets Ok Computer...though not as perfec...,1106697600,Alternative Rock,$7.98,p24253921,6131,18153613,6,136
77389,5,"03 9, 2015",u17905699,12975,perfect,Five Stars,1425859200,Alternative Rock,$8.42,p40118584,10128,47177023,2,46
52734,5,"07 13, 2015",u49280345,35793,very good,Five Stars,1436745600,Pop,$8.47,p80917931,20701,73695126,1,134
80652,5,"07 13, 2014",u19712170,14280,OK well let me start off the shipping was very...,The Legend great cd,1405209600,Jazz,$19.99,p42347694,10690,64861768,1,5
132969,5,"12 14, 2014",u05972617,4357,Love Bastille,Love!,1418515200,Alternative Rock,$22.10,p20950427,5321,4481587,4,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125465,5,"07 6, 2004",u90733150,65676,When I first bought this cd and listened to it...,Not good at first listen but now it is spectac...,1089072000,Alternative Rock,$29.38,p32732560,8277,26360511,4,17
86636,5,"03 3, 2017",u94986584,68682,This has all the big hits!,Five Stars,1488499200,Pop,$7.29,p72914111,18746,51017609,2,47
92444,5,"05 13, 2013",u65828456,47756,I love this album and she came into the scene ...,I love the women singers,1368403200,Pop,$5.99,p41759595,10537,5643233,2,126
21450,4,"11 20, 2013",u06424043,4647,I have never been a huge Daughtry fan but I ha...,Give it a chance if you like good melodies,1384905600,Pop,$9.49,p88386911,22553,44012029,5,45


In [13]:
test_short.to_json(r'./p2TestingShortv2.json', orient='records', lines=True)
train_short.to_json(r'./p2TrainingShortv2.json', orient='records', lines=True)

In [14]:
df_train = df_train.append(train_short)
df_train

,overall,reviewTime,reviewerID,reviewerID_int,reviewText,summary,unixReviewTime,category,price,itemID,itemID_int,reviewHash,reviewerID_int_count,itemID_int_count
0,4,"08 24, 2010",u04428712,3227,"So is Katy Perry's new album ""Teenage Dream"" c...",Amazing that I Actually Bought This...More Ama...,1282608000,Pop,$35.93,p70761125,18183,85559980,11,70
26,2,"09 6, 2006",u01301388,920,"Well, I know that Ministry's career was/is sup...",We've heard it before...,1157500800,Alternative Rock,$2.99,p07016047,1822,73559038,43,10
32,5,"11 3, 2010",u42004673,30557,"Four discs 74,73,49,43 minutes each approximat...",SO MUCH FINE MUSIC WHERE DO YOU START?,1288742400,Pop,$29.31,p68214195,17540,84832082,60,5
35,5,"10 28, 2002",u31093491,22563,"""Kick"" sold more copies, but ""Listen Like Thie...",XS-ively Good.,1035763200,Alternative Rock,$9.99,p70126988,18035,98962224,134,20
40,5,"05 16, 2014",u16807919,12143,"Lou Reed, Berlin. Easily one of the best album...",Lou Reed - Berlin,1400198400,Alternative Rock,$6.64,p49109709,12492,28053917,19,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163153,1,"09 6, 2005",u68040214,49320,this cd didn't click with me. nothing special ...,iron maiden far from worst cd,1125964800,Alternative Rock,$10.12,p32658052,8255,26962294,3,83
132172,3,"10 22, 2014",u31456612,22841,This one pales in comparison to their first al...,Pales in comparison to their first album,1413936000,Pop,$11.09,p05453703,1413,16596252,1,33
1084,5,"06 24, 2006",u91296092,66076,"Call me a poseur, but I did not hook onto AFI'...",Better than Sing the Sorrow?,1151107200,Alternative Rock,$8.95,p38618485,9766,85325140,3,40
20133,4,"05 14, 2017",u18412756,13342,Ok,Ok,1494720000,Alternative Rock,$14.98,p84142912,21498,14204452,2,5


### Figure out unique number of user/items/ratings

In [ ]:
n_users = train['reviewerID_int'].unique().shape[0]
n_items = train['itemID_int'].unique().shape[0]
n_rating = train['overall'].unique().shape[0]

print ('%i unique users' %n_users)
print ('%i unique items' %n_items)
print ('%i unique ratings' %n_rating)

# Loading processed data

In [3]:
df_val = pd.read_json('./p2TestingLongv2.json', lines=True)
df_train = pd.read_json('./p2TrainingLongv2.json', lines=True)
test_short = pd.read_json('./p2TestingShortv2.json', lines=True)
train_short = pd.read_json('./p2TrainingShortv2.json', lines=True)

In [4]:
df_train = df_train.append(train_short)
df_train

,overall,reviewTime,reviewerID,reviewerID_int,reviewText,summary,unixReviewTime,category,price,itemID,itemID_int,reviewHash,reviewerID_int_count,itemID_int_count
0,4,"08 24, 2010",u04428712,3227,"So is Katy Perry's new album ""Teenage Dream"" c...",Amazing that I Actually Bought This...More Ama...,1282608000,Pop,$35.93,p70761125,18183,85559980,11,70
1,2,"09 6, 2006",u01301388,920,"Well, I know that Ministry's career was/is sup...",We've heard it before...,1157500800,Alternative Rock,$2.99,p07016047,1822,73559038,43,10
2,5,"11 3, 2010",u42004673,30557,"Four discs 74,73,49,43 minutes each approximat...",SO MUCH FINE MUSIC WHERE DO YOU START?,1288742400,Pop,$29.31,p68214195,17540,84832082,60,5
3,5,"10 28, 2002",u31093491,22563,"""Kick"" sold more copies, but ""Listen Like Thie...",XS-ively Good.,1035763200,Alternative Rock,$9.99,p70126988,18035,98962224,134,20
4,5,"05 16, 2014",u16807919,12143,"Lou Reed, Berlin. Easily one of the best album...",Lou Reed - Berlin,1400198400,Alternative Rock,$6.64,p49109709,12492,28053917,19,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102595,1,"09 6, 2005",u68040214,49320,this cd didn't click with me. nothing special ...,iron maiden far from worst cd,1125964800,Alternative Rock,$10.12,p32658052,8255,26962294,3,83
102596,3,"10 22, 2014",u31456612,22841,This one pales in comparison to their first al...,Pales in comparison to their first album,1413936000,Pop,$11.09,p05453703,1413,16596252,1,33
102597,5,"06 24, 2006",u91296092,66076,"Call me a poseur, but I did not hook onto AFI'...",Better than Sing the Sorrow?,1151107200,Alternative Rock,$8.95,p38618485,9766,85325140,3,40
102598,4,"05 14, 2017",u18412756,13342,Ok,Ok,1494720000,Alternative Rock,$14.98,p84142912,21498,14204452,2,5


In [5]:
# Only active this when doing alternative test
# df_val = df_val.append(test_short)

# Training the model

In [6]:
# training variables
user_ids = np.array(df_train['reviewerID_int'].tolist())
item_ids = np.array(df_train['itemID_int'].tolist())
user_ratings = np.array(df_train['overall'].tolist())

print(user_ids)
print(len(user_ids))

#Validation variables
user_ids_val = np.array(df_val['reviewerID_int'].tolist())
item_ids_val = np.array(df_val['itemID_int'].tolist())
user_ratings_val = np.array(df_val['overall'].tolist())

[ 3227   920 30557 ... 66076 13342 39637]
135781


In [7]:
# The code is from https://github.com/saurabhmathur96/movie-recommendations/blob/master/notebooks/Recommending%20movies%20using%20SVD%20Matrix%20Factorisation.ipynb
graph = tf.Graph()
n_item = 25494
n_user = 72286
embedding_size = 20

lr = 0.001
reg = 0.05

with graph.as_default():
    user = tf.placeholder(tf.int32, name="user_id") 
    item = tf.placeholder(tf.int32, name="item_id") 
    rating = tf.placeholder(tf.float32, name="rating") 

    item_embedding = tf.Variable(tf.truncated_normal([n_item, embedding_size], stddev=0.02, mean=0.) ,name="item_embedding")
    user_embedding = tf.Variable(tf.truncated_normal([n_user, embedding_size], stddev=0.02, mean=0.) ,name="user_embedding")
    
    item_bias_embedding = tf.Variable(tf.truncated_normal([n_item], stddev=0.02, mean=0.) ,name="item_bias_embedding")
    user_bias_embedding = tf.Variable(tf.truncated_normal([n_user], stddev=0.02, mean=0.) ,name="user_bias_embedding")
    
    
    global_bias = tf.Variable(tf.truncated_normal([], stddev=0.02, mean=0.) ,name="global_bias")
    
    u = tf.nn.embedding_lookup(user_embedding, user)
    m = tf.nn.embedding_lookup(item_embedding, item)
    
    u_bias = tf.nn.embedding_lookup(user_bias_embedding, user)
    m_bias = tf.nn.embedding_lookup(item_bias_embedding, item)
    

    predicted_rating = tf.reduce_sum(tf.math.multiply(u, m), 1) + u_bias + m_bias + global_bias

    mse = tf.reduce_mean(tf.square(predicted_rating - rating)) # MSE
    cost = tf.nn.l2_loss(predicted_rating - rating)
    regularization = reg * (tf.nn.l2_loss(item_embedding) + tf.nn.l2_loss(user_embedding)
                            + tf.nn.l2_loss(item_bias_embedding) + tf.nn.l2_loss(user_bias_embedding))
    
    loss = cost + regularization
    
    optimizer = tf.train.AdamOptimizer(lr).minimize(loss)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
batch_size = 1000
n_epoch = 50


with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    for _ in range(n_epoch):
        for start in range(0, user_ratings.shape[0] - batch_size, batch_size):
            end = start + batch_size
            _, cost_value = sess.run([optimizer, mse], feed_dict={user: user_ids[start:end],
                                                  item: item_ids[start: end],
                                                  rating: user_ratings[start: end]})

        print ("MSE", cost_value)
    embeddings = item_embedding.eval()
        
    print("Starting validation set")
    predict_val = sess.run(predicted_rating, feed_dict={user: user_ids_val[:],
                                                  item: item_ids_val[:],
                                                  rating: user_ratings_val[:]})
    
    mse_val = mean_squared_error(user_ratings_val, predict_val)
    print ("predict_val", mse_val)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
MSE 18.848114
MSE 17.224455
MSE 15.660883
MSE 14.08645
MSE 12.492848
MSE 10.930225
MSE 9.465329
MSE 8.152417
MSE 7.018772
MSE 6.0641866
MSE 5.2709746
MSE 4.6149282
MSE 4.0719657
MSE 3.620921
MSE 3.2441998
MSE 2.9275327
MSE 2.6594894
MSE 2.4309466
MSE 2.2346272
MSE 2.0647197
MSE 1.9165821
MSE 1.7864913
MSE 1.6714623
MSE 1.5690905
MSE 1.4774494
MSE 1.3949839
MSE 1.3204409
MSE 1.2528133
MSE 1.191299
MSE 1.1352491
MSE 1.0841501
MSE 1.0375851
MSE 0.99522656
MSE 0.9567982
MSE 0.9220724
MSE 0.8908484
MSE 0.8629388
MSE 0.8381613
MSE 0.81632894
MSE 0.7972412
MSE 0.7806892
MSE 0.766449
MSE 0.7542889
MSE 0.74397284
MSE 0.73526686
MSE 0.7279456
MSE 0.7217975
MSE 0.71663064
MSE 0.7122751
MSE 0.7085838
Starting validation set
predict_val 0.85716304129755


In [9]:
completeLabel = np.append(user_ratings_val, test_short.overall.values)

In [10]:
completePred = np.append(predict_val, test_short.predict.values)

In [11]:
mean_squared_error(completeLabel, completePred)

0.48510965754744284